
# Workflow: Raw to JPG
Developed by 2Pi Software <br>
Referencing: <br>
 https://www.numbercrunch.de/blog/2020/12/from-numbers-to-images-raw-image-processing-with-python/ <br>
 https://github.com/letmaik/rawpy-notebooks <br>
 

# Set up libraries
Note! (py)exifTool needs exiftool installed as CLI for support of batch/bindings
see: https://exiftool.org/install.html 

In [ ]:
import os
import matplotlib.pyplot as plt
import rawpy
import exifread
import exiftool
import imageio

# Prove we can read RAW specs of a sample file

In [ ]:
raw_filename = '/whereAreMyFiles'
 
with rawpy.imread(raw_filename) as raw:
    print(f'raw type:                     {raw.raw_type}')                      # raw type (flat or stack, e.g., Foveon sensor)
    print(f'number of colors:             {raw.num_colors}')                    # number of different color components, e.g., 3 for common RGB Bayer sensors with two green identical green sensors 
    print(f'color description:            {raw.color_desc}')                    # describes the various color components
    print(f'raw pattern:                  {raw.raw_pattern.tolist()}')          # decribes the pattern of the Bayer sensor
    print(f'black levels:                 {raw.black_level_per_channel}')       # black level correction
    print(f'white level:                  {raw.white_level}')                   # camera white level
    print(f'color matrix:                 {raw.color_matrix.tolist()}')         # camera specific color matrix, usually obtained from a list in rawpy (not from the raw file)
    print(f'XYZ to RGB conversion matrix: {raw.rgb_xyz_matrix.tolist()}')       # camera specific XYZ to camara RGB conversion matrix
    print(f'camera white balance:         {raw.camera_whitebalance}')           # the picture's white balance as determined by the camera
    print(f'daylight white balance:       {raw.daylight_whitebalance}')         # the camera's daylight white balance

# If RAW was good, check overall health of EXIF

In [ ]:
with open(raw_filename, 'rb') as f:
    tags = exifread.process_file(f)
    for key, value in tags.items():
        if key != 'JPEGThumbnail':  # do not print (uninteresting) binary thumbnail data
            print(f'{key}: {value}')

# Now, appraise conversion options
Throw some previews, in case conversion reveals exceptions, or requires colour space adjustment  
(Try to avoid 'special' settings, as could compromise conformance between multiple batches &/or terrain extremes)  

Stick to : camera whitebalance & no auto brightness, as most stable/repeatable setting!

In [ ]:
with rawpy.imread(raw_filename) as raw:
    image = raw.raw_image
    rgb_wb = raw.postprocess(rawpy.Params(use_camera_wb=True, no_auto_bright=True))
    rgb_ab = raw.postprocess(rawpy.Params(use_auto_wb=True, no_auto_bright=True ))
    rgb_ft = raw.postprocess(rawpy.Params(use_camera_wb=True, auto_bright_thr=0.025))
    rgb_px = raw.postprocess(rawpy.Params(gamma=(16,4), use_camera_wb=True, no_auto_bright=True))
plt.rcParams['figure.figsize'] = [22, 22]
plt.subplot(2,2,1)
plt.axis('off')
plt.imshow(rgb_wb)
plt.subplot(2,2,2)
plt.axis('off')
plt.imshow(rgb_ab)
plt.subplot(2,2,3)
plt.axis('off')
plt.imshow(rgb_ft)
plt.subplot(2,2,4)
plt.axis('off')
plt.imshow(rgb_px)

# Iterate RAW file conversion, using processing per previous step
Transcribe to 'postprocess' parameters below if 'special' processing was determined!

In [ ]:
raw_foldername = "/whereAreMyFiles"

for filename in os.listdir(raw_foldername):
    if filename.endswith('.DNG'):
        with rawpy.imread(os.path.join(raw_foldername, filename)) as raw:
            rgb_wb = raw.postprocess(rawpy.Params(use_camera_wb=True, no_auto_bright=True))            
        base_name = os.path.splitext(filename)[0]
        print(base_name)
        imageio.imsave(f'{raw_foldername}/{base_name}.jpeg', rgb_wb)
        

# Iterate transfer of EXIF from originals to processed files

In [ ]:
raw_foldername = "/whereAreMyFiles"
  
# use lib bindings to do like: exiftool -TagsFromFile Nikon.nef Nikon.tif
for filename in os.listdir(raw_foldername):
    if filename.endswith('.DNG'):           
        base_name = os.path.splitext(filename)[0]
        src = f'-{raw_foldername}/{base_name}.DNG'
        dst = f'-{raw_foldername}/{base_name}.jpeg'
        print(src + " : " + dst)
        with exiftool.ExifTool() as et:
            et.execute("-TagsFromFile", src, dst)

# Finally, spot check on a conversion for health and EXIF transcribed

In [ ]:
jpg_filename = 'chooseATypicalFile.jpeg'

with open(raw_filename, 'rb') as f:
    tags = exifread.process_file(f)
    for key, value in tags.items():
        if key != 'JPEGThumbnail':  # do not print (uninteresting) binary thumbnail data
            print(f'{key}: {value}')